In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import sys
import os

nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)
sys.path.insert(0, "./")
sys.path.insert(0, "/dadendev/nlp/examples/text_summarization/BertSum/")

In [4]:
print(sys.path)

['/dadendev/nlp/examples/text_summarization/BertSum/', './', '/dadendev/nlp', '/dadendev/anaconda3/envs/cm3/lib/python36.zip', '/dadendev/anaconda3/envs/cm3/lib/python3.6', '/dadendev/anaconda3/envs/cm3/lib/python3.6/lib-dynload', '', '/home/daden/.local/lib/python3.6/site-packages', '/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages', '/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages/pyrouge-0.1.3-py3.6.egg', '/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages/IPython/extensions', '/home/daden/.ipython']


In [5]:
#"""
from utils_nlp.dataset.cnndm import CNNDMSummarization

train_dataset, test_dataset = CNNDMSummarization(top_n=5)

len(train_dataset)

len(test_dataset)

test_dataset[0]
#"""

[nltk_data] Downloading package punkt to /home/daden/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
0lines [00:00, ?lines/s]
0lines [00:00, ?lines/s]
0lines [00:00, ?lines/s]
0lines [00:00, ?lines/s]


[['marseille',
  ',',
  'france',
  '(',
  'cnn',
  ')',
  'the',
  'french',
  'prosecutor',
  'leading',
  'an',
  'investigation',
  'into',
  'the',
  'crash',
  'of',
  'germanwings',
  'flight',
  '9525',
  'insisted',
  'wednesday',
  'that',
  'he',
  'was',
  'not',
  'aware',
  'of',
  'any',
  'video',
  'footage',
  'from',
  'on',
  'board',
  'the',
  'plane',
  '.'],
 ['marseille',
  'prosecutor',
  'brice',
  'robin',
  'told',
  'cnn',
  'that',
  '``',
  'so',
  'far',
  'no',
  'videos',
  'were',
  'used',
  'in',
  'the',
  'crash',
  'investigation',
  '.',
  '``'],
 ['he',
  'added',
  ',',
  '``',
  'a',
  'person',
  'who',
  'has',
  'such',
  'a',
  'video',
  'needs',
  'to',
  'immediately',
  'give',
  'it',
  'to',
  'the',
  'investigators',
  '.',
  '``'],
 ['robin',
  "'s",
  'comments',
  'follow',
  'claims',
  'by',
  'two',
  'magazines',
  ',',
  'german',
  'daily',
  'bild',
  'and',
  'french',
  'paris',
  'match',
  ',',
  'of',
  'a',
  'cel

In [6]:
from utils_nlp.models.transformers.extractive_summarization import ExtSumProcessor, ExtractiveSummarizer

I1028 19:36:39.891576 139886807013184 file_utils.py:39] PyTorch version 1.2.0 available.
I1028 19:36:39.924971 139886807013184 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
I1028 19:36:39.943137 139886807013184 modeling.py:230] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [7]:
processor = ExtSumProcessor()

I1028 19:36:40.130957 139886807013184 tokenization_utils.py:374] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at ./5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


In [8]:
#"""
ext_sum_train = processor.preprocess(list(train_dataset), list(train_dataset.get_target()))

ext_sum_test = processor.preprocess(list(test_dataset), list(test_dataset.get_target()))

#type(ext_sum_train)
#"""

In [9]:
import glob
BERT_DATA_PATH = BERT_DATA_PATH="./bert_data/"
pts = sorted(glob.glob(BERT_DATA_PATH + 'cnndm.train' + '.[0-9]*.pt'))

In [10]:
pts[0:2]

['./bert_data/cnndm.train.0.bert.pt', './bert_data/cnndm.train.1.bert.pt']

In [11]:
import torch
def get_dataset(file_list):
    #random.shuffle(file_list)
    for file in file_list:
        yield torch.load(file)

In [12]:
from bertsum.models.data_loader  import DataIterator
from bertsum.models import  model_builder, data_loader
class Bunch(object):
    """ Class which convert a dictionary to an object """

    def __init__(self, adict):
        self.__dict__.update(adict)
def get_data_loader(data_files, device, is_labeled=False, batch_size=3000):
    """
    Function to get data iterator over a list of data objects.

    Args:
        dataset (list of objects): a list of data objects.
        is_test (bool): it specifies whether the data objects are labeled data.
        batch_size (int): number of tokens per batch.
        
    Returns:
        DataIterator

    """
    args = Bunch({})
    args.use_interval = True
    args.batch_size = batch_size
    data_iter = None
    data_iter  = data_loader.Dataloader(args, get_dataset(data_files), args.batch_size,device, shuffle=False, is_test=is_labeled)
    return data_iter

In [13]:
device = torch.device("cuda:{}".format(0)) 
def train_iter_func():
    return get_data_loader(pts, device, is_labeled=True)

In [14]:
from transformers import PreTrainedModel,  PretrainedConfig, DistilBertModel, BertModel, DistilBertConfig
summarizer = None
summarizer = ExtractiveSummarizer()

I1028 19:36:40.696836 139886807013184 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at ./a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.1ccd1a11c9ff276830e114ea477ea2407100f4a3be7bdc45d37be9e37fa71c7e
I1028 19:36:40.697976 139886807013184 configuration_utils.py:168] Model config {
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": null,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "n_heads": 12,
  "n_layers": 6,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torchscript": false,
  "use_bfloat16": false,
  "vocab_size": 30522
}

I1028 19:36:40.822206 139886807013184 modeling_utils.

In [15]:
# notebook parameters
DATA_FOLDER = "./temp"
CACHE_DIR = "./temp"
DEVICE = "cuda"
NUM_EPOCHS = 1
BATCH_SIZE = 64
NUM_GPUS = 1
MAX_LEN = 150

In [16]:
### from utils_nlp.common.timer import Timer

    summarizer.fit(
            train_iter_func,
            device= DEVICE,
            num_epochs=NUM_EPOCHS,
            batch_size=BATCH_SIZE,
            num_gpus=NUM_GPUS,
            max_steps=1e4,
            learning_rate=1e-4,
            warmup_steps=1e4*0.5,
            verbose=True,
            report_every=100,
        )
  
    


0
loss: 0.434776, time: 0.33, step 0.000000 out of total 10000.000000
loss: 52.902908, time: 11.08, step 100.000000 out of total 10000.000000
loss: 38.612158, time: 11.01, step 200.000000 out of total 10000.000000
loss: 35.420277, time: 11.03, step 300.000000 out of total 10000.000000
loss: 34.828366, time: 11.95, step 400.000000 out of total 10000.000000
loss: 34.409282, time: 11.49, step 500.000000 out of total 10000.000000
loss: 33.686915, time: 11.44, step 600.000000 out of total 10000.000000
loss: 33.543083, time: 11.21, step 700.000000 out of total 10000.000000
loss: 33.193735, time: 11.82, step 800.000000 out of total 10000.000000
loss: 32.814421, time: 11.23, step 900.000000 out of total 10000.000000
loss: 32.306093, time: 11.26, step 1000.000000 out of total 10000.000000
loss: 32.094086, time: 11.12, step 1100.000000 out of total 10000.000000
loss: 33.199048, time: 11.75, step 1200.000000 out of total 10000.000000
loss: 31.443077, time: 11.44, step 1300.000000 out of total 100

In [17]:
torch.save(summarizer.model, "cnndm_transformersum_distilbert-base-uncased_bertsum_processed_data.pt")

In [24]:
import torch
from utils_nlp.models.bert.extractive_text_summarization import get_data_iter
import os

test_dataset=[]
for i in range(0,6):
    filename = os.path.join(BERT_DATA_PATH, "cnndm.test.{0}.bert.pt".format(i))
    test_dataset.extend(torch.load(filename))

In [25]:
prediction = summarizer.predict(get_data_iter(test_dataset),
                               device=DEVICE,)

In [26]:
target = [test_dataset[i]['tgt_txt'] for i in range(len(test_dataset))]

In [27]:
from utils_nlp.eval.evaluate_summarization import get_rouge
rouge_baseline = get_rouge(prediction, target, "./results/")

11489
11489


2019-10-28 20:05:12,147 [MainThread  ] [INFO ]  Writing summaries.
I1028 20:05:12.147566 139886807013184 pyrouge.py:525] Writing summaries.
2019-10-28 20:05:12,149 [MainThread  ] [INFO ]  Processing summaries. Saving system files to ./results/tmpnapu1ceb/system and model files to ./results/tmpnapu1ceb/model.
I1028 20:05:12.149479 139886807013184 pyrouge.py:518] Processing summaries. Saving system files to ./results/tmpnapu1ceb/system and model files to ./results/tmpnapu1ceb/model.
2019-10-28 20:05:12,150 [MainThread  ] [INFO ]  Processing files in ./results/rouge-tmp-2019-10-28-20-05-11/candidate/.
I1028 20:05:12.150503 139886807013184 pyrouge.py:43] Processing files in ./results/rouge-tmp-2019-10-28-20-05-11/candidate/.
2019-10-28 20:05:13,204 [MainThread  ] [INFO ]  Saved processed files to ./results/tmpnapu1ceb/system.
I1028 20:05:13.204638 139886807013184 pyrouge.py:53] Saved processed files to ./results/tmpnapu1ceb/system.
2019-10-28 20:05:13,206 [MainThread  ] [INFO ]  Processing

---------------------------------------------
1 ROUGE-1 Average_R: 0.53918 (95%-conf.int. 0.53643 - 0.54191)
1 ROUGE-1 Average_P: 0.36463 (95%-conf.int. 0.36234 - 0.36693)
1 ROUGE-1 Average_F: 0.42091 (95%-conf.int. 0.41880 - 0.42302)
---------------------------------------------
1 ROUGE-2 Average_R: 0.24454 (95%-conf.int. 0.24200 - 0.24735)
1 ROUGE-2 Average_P: 0.16532 (95%-conf.int. 0.16341 - 0.16740)
1 ROUGE-2 Average_F: 0.19061 (95%-conf.int. 0.18852 - 0.19274)
---------------------------------------------
1 ROUGE-L Average_R: 0.49144 (95%-conf.int. 0.48881 - 0.49416)
1 ROUGE-L Average_P: 0.33284 (95%-conf.int. 0.33057 - 0.33513)
1 ROUGE-L Average_F: 0.38398 (95%-conf.int. 0.38192 - 0.38601)



In [22]:
prediction[0]

'andrew mogni , 20 , from glen ellyn , illinois , had only just arrived for a semester program in italy when the incident happened in january .<q>he was flown back to chicago via air ambulance on march 20 , but he died on sunday .<q>a university of iowa student has died nearly three months after a fall in rome in a suspected robbery attack in rome .'

In [23]:
test_dataset[0]['tgt_txt']

'andrew mogni , 20 , from glen ellyn , illinois , had only just arrived for a semester program when the incident happened in january<q>he was flown back to chicago via air on march 20 but he died on sunday<q>initial police reports indicated the fall was an accident but authorities are investigating the possibility that mogni was robbed<q>his cousin claims he was attacked and thrown 40ft from a bridge'